In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedKFold
from sklearn.metrics import r2_score
import numpy as np
from ipywidgets import interactive, IntSlider, VBox, Label, Layout, fixed
from IPython.display import display

def plot_model_predictions_over_days():
    """
    This function is a placeholder for future implementation.
    """
    raise NotImplementedError("This function is not yet implemented.")

%run example_definitions/lecture_06/plot_phenotyping_regression.ipynb

def linear_regression(df_loaded):
    model_name = "Linear Regression"
    model = LinearRegression()

    # ----------------------
    # Define Features/Target
    # ----------------------
    feature_cols = [
        "days_of_phenotyping",
        "species",
        "nitrogen_applied",
        "drought_stress",
    ]
    target_col = "digital_biomass"

    X_data_init = df_loaded[feature_cols]
    y_data_init = df_loaded[target_col]

    # --------------------------
    # Train/Test Split
    # --------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        X_data_init, y_data_init, test_size=0.3, random_state=42
    )

    df_train = X_train.copy()
    df_train[target_col] = y_train

    # -----------------------
    # Define Column Groups
    # -----------------------
    categorical_features = ["species"]
    numeric_features = [
        "days_of_phenotyping",
        "nitrogen_applied",
        "drought_stress",
    ]

    # --------------------------
    # Build Preprocessing Pipeline
    # --------------------------
    preprocessor = ColumnTransformer(
        [
            (
                "onehot_species",
                Pipeline(
                    [
                        ("onehot", OneHotEncoder(drop="first")),
                        ("scaler", StandardScaler(with_mean=False)),  # Important: with_mean=False for sparse data
                    ]
                ),
                categorical_features,
            ),
            ("num", StandardScaler(), numeric_features),
        ]
    )

    _ = preprocessor.fit_transform(X_data_init)  # Fit preprocessor on entire data
    pipeline = Pipeline([("preprocess", preprocessor), ("regressor", model)])

    # %%
    # --------------------------
    # 6b. Scale target variable (y)
    # --------------------------
    y_scaler = StandardScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()

    # --------------------------
    # 7. Fit Model
    # --------------------------
    pipeline.fit(X_train, y_train_scaled)

    # --------------------------
    # 8. Evaluate
    # --------------------------

    #model = pipeline.named_steps["regressor"]
    # Predict scaled targets on test set
    y_pred_scaled = pipeline.predict(X_test)

    # Inverse transform predictions back to original scale
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

    r2_test_ridge = r2_score(y_test, y_pred)
    mse_test_ridge = np.mean((y_test - y_pred) ** 2)
    rmse_test_ridge = np.sqrt(mse_test_ridge)

    print("--------------------------------------------------")
    print(f"{model_name} - Regression Results:")
    print(f"Test Set RMSE: {rmse_test_ridge:.3e}")
    print(f"Test Set R² score: {r2_test_ridge:.3f}")
    print("--------------------------------------------------")

    # %% Test predictions over days for different nitrogen levels
    plot_model_predictions_over_days(
        preprocessor,
        y_scaler,
        model,
        model_name,
        df_loaded,
    )

def linear_reg_interact(df_loaded):
    interactive_plot = interactive(
        linear_regression,
        df_loaded=fixed(df_loaded),
    )

    display(interactive_plot)